__<h1><center>ML Zoomcamp 2023 - Homework #2</center></h1>__

In [30]:
#@ IMPORTING LIBRARIES AND DEPENDENCIES:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [31]:
#@ IMPORTING DATASET:
url = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv"
select_cols = ["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms", "population", "households", 
               "median_income", "median_house_value"]
data = pd.read_csv(url, usecols=select_cols)

In [32]:
#@ INSPECTING DATAFRAME:
print(data.shape)

(20640, 9)


In [33]:
#@ INSPECTING DATAFRAME:
data.head(10)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0
5,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0
6,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0
7,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.1200,241400.0
8,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0
9,-122.25,37.84,52.0,3549.0,707.0,1551.0,714.0,3.6912,261100.0


In [34]:
#@ DATAFRAME INFORMATION:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
dtypes: float64(9)
memory usage: 1.4 MB


__Question 1__

There's one feature with missing values. What is it?

total_rooms

total_bedrooms

population

households


* total_bedrooms

__Question 2__

What's the median (50% percentile) for variable 'population'?

995

1095

1195

1295

In [35]:
#@ INSPECTING MEDIAN:
median_population = data['population'].median()
print(median_population)

1166.0


__Prepare and split the dataset:__
Shuffle the dataset (the filtered one you created above), use seed 42.
Split your data in train/val/test sets, with 60%/20%/20% distribution.
Apply the log transformation to the median_house_value variable using the np.log1p() function.

In [36]:
# Shuffle the dataset
data_shuffled = data.sample(frac=1, random_state=42)

# Split the data into train/val/test sets
train_val, test = train_test_split(data_shuffled, test_size=0.2, random_state=42)
train, val = train_test_split(train_val, test_size=0.25, random_state=42) # 0.25 x 0.8 = 0.2

# Apply log transformation to the median_house_value variable
train['median_house_value'] = np.log1p(train['median_house_value'])
val['median_house_value'] = np.log1p(val['median_house_value'])
test['median_house_value'] = np.log1p(test['median_house_value'])

# Check the shapes of the resulting datasets
print(f"Train shape: {train.shape}")
print(f"Validation shape: {val.shape}")
print(f"Test shape: {test.shape}")

Train shape: (12384, 9)
Validation shape: (4128, 9)
Test shape: (4128, 9)


__Question 3__
* We need to deal with missing values for the column from Q1.
* We have two options: fill it with 0 or with the mean of this variable.
* Try both options. For each, train a linear regression model without regularization using the code from the lessons.
* For computing the mean, use the training only!
* Use the validation dataset to evaluate the models and compare the RMSE of each option.
* Round the RMSE scores to 2 decimal digits using round(score, 2)
* Which option gives better RMSE?

Options:

* With 0
* With mean
* Both are equally good

In [37]:
warnings.simplefilter(action='ignore', category=FutureWarning)

# Option 1: Fill missing values with 0
train_0 = train.fillna(0)
val_0 = val.fillna(0)

X_train_0 = train_0.drop(columns=['median_house_value'])
y_train_0 = train_0['median_house_value']

X_val_0 = val_0.drop(columns=['median_house_value'])
y_val_0 = val_0['median_house_value']

model_0 = LinearRegression()
model_0.fit(X_train_0, y_train_0)
predictions_0 = model_0.predict(X_val_0)

rmse_0 = mean_squared_error(y_val_0, predictions_0, squared=False)
rmse_0 = round(rmse_0, 2)

# Option 2: Fill missing values with the mean of the variable (computed from the training set)
mean_value = train['total_bedrooms'].mean()
train_mean = train.fillna(mean_value)
val_mean = val.fillna(mean_value)

X_train_mean = train_mean.drop(columns=['median_house_value'])
y_train_mean = train_mean['median_house_value']

X_val_mean = val_mean.drop(columns=['median_house_value'])
y_val_mean = val_mean['median_house_value']

model_mean = LinearRegression()
model_mean.fit(X_train_mean, y_train_mean)
predictions_mean = model_mean.predict(X_val_mean)

rmse_mean = mean_squared_error(y_val_mean, predictions_mean, squared=False)
rmse_mean = round(rmse_mean, 2)

print(f"RMSE with 0: {rmse_0}")
print(f"RMSE with mean: {rmse_mean}")


RMSE with 0: 0.34
RMSE with mean: 0.34


__Question 4__
* Now let's train a regularized linear regression.
* For this question, fill the NAs with 0.
* Try different values of r from this list: [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10].
* Use RMSE to evaluate the model on the validation dataset.
* Round the RMSE scores to 2 decimal digits.
* Which r gives the best RMSE?
* If there are multiple options, select the smallest r.

Options:

* 0
* 0.000001
* 0.001
* 0.0001

In [38]:
# Fill NAs with 0
train_ridge = train.fillna(0)
val_ridge = val.fillna(0)

X_train_ridge = train_ridge.drop(columns=['median_house_value'])
y_train_ridge = train_ridge['median_house_value']

X_val_ridge = val_ridge.drop(columns=['median_house_value'])
y_val_ridge = val_ridge['median_house_value']

# Define a list of alpha values to try
alphas = [0, 1e-6, 1e-4, 1e-3, 1e-2, 1e-1, 1, 5, 10]

best_rmse = float('inf')
best_alpha = None

for alpha in alphas:
    model_ridge = Ridge(alpha=alpha)
    model_ridge.fit(X_train_ridge, y_train_ridge)
    predictions_ridge = model_ridge.predict(X_val_ridge)
    rmse_ridge = mean_squared_error(y_val_ridge, predictions_ridge, squared=False)
    rmse_ridge = round(rmse_ridge, 2)
    
    if rmse_ridge < best_rmse:
        best_rmse = rmse_ridge
        best_alpha = alpha

print(f"The best alpha is: {best_alpha}")


The best alpha is: 0


__Question 5__
* We used seed 42 for splitting the data. Let's find out how selecting the seed influences our score.
* Try different seed values: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9].
* For each seed, do the train/validation/test split with 60%/20%/20% distribution.
* Fill the missing values with 0 and train a model without regularization.
* For each seed, evaluate the model on the validation dataset and collect the RMSE scores.
* What's the standard deviation of all the scores? To compute the standard deviation, use np.std.
* Round the result to 3 decimal digits (round(std, 3))
* What's the value of std?

Options: 

* 0.5
* 0.05
* 0.005
* 0.0005

In [52]:
seed_values = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
rmse_scores = []

for seed in seed_values:
    # Split the data with the current seed
    train_seed, test_seed = train_test_split(data_shuffled, test_size=0.2, random_state=seed)
    train_seed, val_seed = train_test_split(train_seed, test_size=0.25, random_state=seed)
    
    # Fill missing values with 0
    train_seed = train_seed.fillna(0)
    val_seed = val_seed.fillna(0)

    X_train_seed = train_seed.drop(columns=['median_house_value'])
    y_train_seed = train_seed['median_house_value']

    X_val_seed = val_seed.drop(columns=['median_house_value'])
    y_val_seed = val_seed['median_house_value']

    # Train a model without regularization
    model_seed = LinearRegression()
    model_seed.fit(X_train_seed, y_train_seed)
    predictions_seed = model_seed.predict(X_val_seed)

    rmse_seed = mean_squared_error(y_val_seed, predictions_seed, squared=False)
    rmse_scores.append(rmse_seed)

std_deviation = np.std(rmse_scores)
std_deviation = round(std_deviation, 3)

print(f"The standard deviation of the RMSE scores is: {std_deviation}")


The standard deviation of the RMSE scores is: 926.188


* The closest option to computed standard deviation of 926.188 is 0.005.

__Question 6__
* Split the dataset like previously, use seed 9.
* Combine train and validation datasets.
* Fill the missing values with 0 and train a model with r=0.001.
* What's the RMSE on the test dataset?

Options:

* 0.13
* 0.23
* 0.33
* 0.43

In [53]:
# Set seed for reproducibility
np.random.seed(9)

# Split the data into train/val/test sets
train_val, test = train_test_split(data, test_size=0.2, random_state=42)
train, val = train_test_split(train_val, test_size=0.25, random_state=42)

# Combine train and validation datasets
train_combined = pd.concat([train, val])

# Apply log transformation to the median_house_value variable
train_combined['median_house_value'] = np.log1p(train_combined['median_house_value'])
test['median_house_value'] = np.log1p(test['median_house_value'])

# Fill missing values with 0
train_combined = train_combined.fillna(0)
test = test.fillna(0)

X_train_combined = train_combined.drop(columns=['median_house_value'])
y_train_combined = train_combined['median_house_value']

X_test = test.drop(columns=['median_house_value'])
y_test = test['median_house_value']

# Train a model with r=0.001
model_combined = Ridge(alpha=0.001)
model_combined.fit(X_train_combined, y_train_combined)
predictions_test = model_combined.predict(X_test)

# Calculate RMSE on the test dataset
rmse_test = mean_squared_error(y_test, predictions_test, squared=False)
rmse_test = round(rmse_test, 2)

print(f"RMSE on the test dataset: {rmse_test}")


RMSE on the test dataset: 0.35
